## Preprocessing

In [ ]:
pip install tensorflow

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [5]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

# Display the result
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [6]:
# Look at APPLICATION_TYPE value counts
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the result
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the result
print(classification_counts)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [9]:
# Filter CLASSIFICATION value counts for counts > 1
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]

# Display the result
print(classification_counts_greater_than_1)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Choose a cutoff value
cutoff_value_classification = 100  # You can adjust this value based on your data exploration

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff_value_classification].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df)

# Display the resulting DataFrame
print(application_df_encoded.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                   False   
1       1   108590              1                   False   
2       1     5000              0                   False   
3       1     6692              1                   False   
4       1   142590              1                   False   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                  True                 False                False   
1                 False                 False                 True   
2                 False                 False                False   
3                 False                 False                 True   
4                 False                 False                 True   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                False                False                False  ...   
1                False                False                False  

In [12]:
# Identify the target column
target_column = 'IS_SUCCESSFUL'

# Separate the features (X) and the target (y)
X = application_df_encoded.drop(columns=[target_column])
y = application_df_encoded[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Display the shapes of the training and testing sets
print("Training set - Features:", X_train.shape, "Target:", y_train.shape)
print("Testing set - Features:", X_test.shape, "Target:", y_test.shape)


Training set - Features: (25724, 49) Target: (25724,)
Testing set - Features: (8575, 49) Target: (8575,)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train.columns)

# First hidden layer
nn.add(tf.keras.layers.Dense(units=8, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=4, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 400       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 441 (1.72 KB)
Trainable params: 441 (1.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50


804/804 [==============================] - 2s 1ms/step - loss: 0.6316 - accuracy: 0.6661
Epoch 2/50
804/804 [==============================] - 1s 962us/step - loss: 0.5721 - accuracy: 0.7262
Epoch 3/50
804/804 [==============================] - 1s 952us/step - loss: 0.5626 - accuracy: 0.7283
Epoch 4/50
804/804 [==============================] - 1s 951us/step - loss: 0.5569 - accuracy: 0.7290
Epoch 5/50
804/804 [==============================] - 1s 975us/step - loss: 0.5539 - accuracy: 0.7306
Epoch 6/50
804/804 [==============================] - 1s 984us/step - loss: 0.5519 - accuracy: 0.7313
Epoch 7/50
804/804 [==============================] - 1s 940us/step - loss: 0.5505 - accuracy: 0.7312
Epoch 8/50
804/804 [==============================] - 1s 966us/step - loss: 0.5492 - accuracy: 0.7318
Epoch 9/50
804/804 [==============================] - 1s 964us/step - loss: 0.5481 - accuracy: 0.7320
Epoch 10/50
804/804 [==============================] - 1s 974us/step - loss: 0.547

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5544 - accuracy: 0.7240 - 309ms/epoch - 1ms/step
Loss: 0.5543743968009949, Accuracy: 0.7239649891853333


In [38]:
# Specify the file path for the HDF5 file
file_path = "your_model.h5"

# Save the model to the specified file path
nn.save(file_path)

In [50]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Define a function for a simple learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Create a Sequential model
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train.columns)

# Define the model architecture
nn.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=input_features))
nn.add(tf.keras.layers.Dropout(0.3))
nn.add(tf.keras.layers.Dense(units=16, activation='tanh'))
nn.add(tf.keras.layers.Dropout(0.2))
nn.add(tf.keras.layers.Dense(units=8, activation='relu'))
nn.add(tf.keras.layers.Dropout(0.2))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model with a lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
nn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Implement early stopping and learning rate scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Train the model
history = nn.fit(
    X_train_scaled, y_train, 
    epochs=50, 
    validation_data=(X_test_scaled, y_test), 
    callbacks=[early_stopping, lr_scheduler_callback]
)

# Evaluate the model on the test set
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6972 - accuracy: 0.5579 - val_loss: 0.6432 - val_accuracy: 0.6630 - lr: 1.0000e-04
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6548 - accuracy: 0.6218 - val_loss: 0.6112 - val_accuracy: 0.7008 - lr: 1.0000e-04
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6332 - accuracy: 0.6591 - val_loss: 0.5930 - val_accuracy: 0.7068 - lr: 1.0000e-04
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6189 - accuracy: 0.6764 - val_loss: 0.5835 - val_accuracy: 0.7068 - lr: 1.0000e-04
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6117 - accuracy: 0.6856 - val_loss: 0.5786 - val_accuracy: 0.7052 - lr: 1.0000e-04
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6049 - accuracy: 0.6941 - val_loss: 0.5748 - val_accuracy: 0.7053 - lr: 1.0000e-04
Epoch 7/50
804/804 [==========================

In [51]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5624 - accuracy: 0.7187 - 216ms/epoch - 807us/step
Loss: 0.5624094605445862, Accuracy: 0.7187172174453735
